In [ ]:
!pip install -q \
    torch==2.1.0 torchvision==0.16.1 \
    pillow opencv-python moviepy tqdm supervision

!git clone --depth 1 https://github.com/roboflow/rf-detr.git
%cd rf-detr
!pip install -q -e .
%cd ..

print("✅ RF-DETR установлен")

----

In [ ]:
from google.colab import drive, files
import torch, cv2, os
import numpy as np
from tqdm.auto import tqdm
from rfdetr import RFDETRBase
import pathlib, sys


In [ ]:
drive.mount('/content/drive')

WEIGHTS_PATH = '/content/drive/MyDrive/Colab Notebooks/rf_detr_803.pt'

assert pathlib.Path(WEIGHTS_PATH).is_file(), f"❌ Файл не найден: {WEIGHTS_PATH}"
print("Чекпойнт найден")


----

In [ ]:
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Используем устройство: {device_str}")

----

In [ ]:
device_str = 'cuda' if torch.cuda.is_available() else 'cpu'

model = RFDETRBase(
    pretrain_weights = WEIGHTS_PATH,
    device           = device_str
)

CLASSES  = {1: 'foul'}
SCORE_TH = 0.5
print(" Модель загружена — готова к predict()")


----

In [5]:
model.optimize_for_inference()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


----

In [ ]:
print("Выберите локальный .mp4")
up = files.upload()
assert len(up) == 1, "Загрузите ровно один файл!"
in_name = next(iter(up))
in_path = f"/content/{in_name}"
print(f"Видео загружено: {in_path}")


----

In [7]:
cap = cv2.VideoCapture(in_path)
fps = cap.get(cv2.CAP_PROP_FPS) or 25
w   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_path = in_path.replace(".mp4", "_annotated.mp4")
writer   = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

print(f"✅ FPS={fps:.1f}, размер={w}×{h}; выход: {out_path}")


✅ FPS=59.9, размер=1920×1080; выход: /content/foull_annotated.mp4


----

In [8]:
from tqdm.auto import tqdm
import cv2

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or None
pbar = tqdm(total=total_frames, desc="Processing")

while True:
    ok, frame_bgr = cap.read()
    if not ok:
        break

    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)


    detections = model.predict(frame_rgb, conf_thres=SCORE_TH)

    for xyxy, conf, cid in zip(detections.xyxy,
                               detections.confidence,
                               detections.class_id):
        if cid != 1:
            continue
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(frame_bgr, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame_bgr, f"foul {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    writer.write(frame_bgr)
    pbar.update(1)

pbar.close()
cap.release(); writer.release()
print(f"Аннотированное видео сохранено: {out_path}")


Processing:   0%|          | 0/501 [00:00<?, ?it/s]

✅ Аннотированное видео сохранено: /content/foull_annotated.mp4


----